<a href="https://colab.research.google.com/github/day-ch/Appteste/blob/main/O_que_se_tem_estudado_sobre_videogames_e_o_espectro_autista%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### O que se tem estudado sobre videogames e o espectro autista? Uma revisão sistemática de dados da base pubmed com auxílio de processamento de linguagem natural e deep learning




Este notebook traz o código em Phython utilizado para a elaboração da revisão sistemática de dados do pubmed "O que se tem estudado sobre videogames e o espectro autista: uma revisão sistemática de dados da base pubmed com auxílio de processamento de linguagem natural e deep learning", apresentado no formato de poster no VII Congresso Internacional e XXVII Congresso Nacional da ABENEPI (Associação Brasileira de Neurologia e Psiquiatria Infantil e Profissões Afins)

Os códigos podem ser utilizados em auxílio a pesquisas e revisões de outros temas na base pubmed.

Para pesquisar no pubmed o tema de interesse (("Autism Spectrum Disorder"[Mesh]) AND "Video Games"[Mesh]), baixar de forma automatizada os resultados da busca no formato de uma tabela (tabela com um artigo por linha com

1) DOWNLOAD DOS ABSTRACTS


> Inicialmente o tema de interesse é buscado na base pubmed e os dados de cada artigo encontrado são baixados e salvos em um tabela no formato '.csv'. São dois arquivos gerados, um com os artigos que tem 'abstracts' completos e um arquivo com 'abstracts' parciais.


> Cada linha da tabela corresponde a um artigo, com colunas para título do artigo, autores, informações de autores, revista, 'abstract', DOI e miscelânea.


> O algoritmo abaixo utiliza o pacote 'eutils 0.6.0' para esses procedimentos.








In [ ]:
import csv
import re
import urllib
from time import sleep

query = '(("Autism+Spectrum+Disorder"%5BMesh%5D)++AND+"Video+Games"%5BMesh%5D'
# common settings between esearch and efetch
base_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
db = 'db=pubmed'

# esearch settings
search_eutil = 'esearch.fcgi?'
search_term = '&term=' + query
search_usehistory = '&usehistory=y'
search_rettype = '&rettype=json'

# call the esearch command for the query and read the web result
search_url = base_url+search_eutil+db+search_term+search_usehistory+search_rettype
print("this is the esearch command:\n" + search_url + "\n")
f = urllib.request.urlopen (search_url)
search_data = f.read().decode('utf-8')

# extract the total abstract count
total_abstract_count = int(re.findall("<Count>(\d+?)</Count>",search_data)[0])

# efetch settings
fetch_eutil = 'efetch.fcgi?'
retmax = 20
retstart = 0
fetch_retmode = "&retmode=text"
fetch_rettype = "&rettype=abstract"

# obtain webenv and querykey settings from the esearch results
fetch_webenv = "&WebEnv=" + re.findall ("<WebEnv>(\S+)<\/WebEnv>", search_data)[0]
fetch_querykey = "&query_key=" + re.findall("<QueryKey>(\d+?)</QueryKey>",search_data)[0]

# call efetch commands using a loop until all abstracts are obtained
run = True
all_abstracts = list()
loop_counter = 1

while run:
    print("this is efetch run number " + str(loop_counter))
    loop_counter += 1
    fetch_retstart = "&retstart=" + str(retstart)
    fetch_retmax = "&retmax=" + str(retmax)
    # create the efetch url
    fetch_url = base_url+fetch_eutil+db+fetch_querykey+fetch_webenv+fetch_retstart+fetch_retmax+fetch_retmode+fetch_rettype
    print(fetch_url)
    # open the efetch url
    f = urllib.request.urlopen (fetch_url)
    fetch_data = f.read().decode('utf-8')
    # split the data into individual abstracts
    abstracts = fetch_data.split("\n\n\n")
    # append to the list all_abstracts
    all_abstracts = all_abstracts+abstracts
    print("a total of " + str(len(all_abstracts)) + " abstracts have been downloaded.\n")
    # wait 2 seconds so we don't get blocked
    sleep(2)
    # update retstart to download the next chunk of abstracts
    retstart = retstart + retmax
    if retstart > total_abstract_count:
        run = False


this is the esearch command:
http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(("Autism+Spectrum+Disorder"%5BMesh%5D)++AND+"Video+Games"%5BMesh%5D&usehistory=y&rettype=json

this is efetch run number 1
http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&query_key=1&WebEnv=MCID_65341f44b2e2fd658c256d30&retstart=0&retmax=20&retmode=text&rettype=abstract
a total of 20 abstracts have been downloaded.

this is efetch run number 2
http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&query_key=1&WebEnv=MCID_65341f44b2e2fd658c256d30&retstart=20&retmax=20&retmode=text&rettype=abstract
a total of 40 abstracts have been downloaded.

this is efetch run number 3
http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&query_key=1&WebEnv=MCID_65341f44b2e2fd658c256d30&retstart=40&retmax=20&retmode=text&rettype=abstract
a total of 60 abstracts have been downloaded.

this is efetch run number 4
http://eutils.ncbi.nlm.nih.gov/entrez/eutils/e

In [ ]:
with open("TCRabstracts.csv", "wt",encoding="utf-8") as abstracts_file:
    abstract_writer = csv.writer(abstracts_file)
    abstract_writer.writerow(['Journal', 'Title', 'Authors', 'Author_Information', 'Abstract', 'DOI', 'Misc'])
    #For each abstract, split into categories and write it to the csv file
    for abstract in all_abstracts:
        #To obtain categories, split every double newline.
        split_abstract = abstract.split("\n\n")
        abstract_writer.writerow(split_abstract)

In [ ]:
with open("TCRabstracts.csv", "wt",encoding="utf-8") as abstracts_file, open ("partial_abstracts.csv", "wt",encoding="utf-8") as partial_abstracts:
    # csv writer for full abstracts
    abstract_writer = csv.writer(abstracts_file)
    abstract_writer.writerow(['Journal', 'Title', 'Authors', 'Author_Information', 'Abstract', 'DOI', 'Misc'])
    # csv writer for partial abstracts
    partial_abstract_writer = csv.writer(partial_abstracts)
    #For each abstract, split into categories and write it to the csv file
    for abstract in all_abstracts:
        #To obtain categories, split every double newline.
        split_abstract = abstract.split("\n\n")
        if len(split_abstract) > 5:
            abstract_writer.writerow(split_abstract)
        else:
            partial_abstract_writer.writerow(split_abstract)

APLICAÇÃO DE CRITÉRIOS DE INCLUSÃO E EXCLUSÃO

A tabela gerada no passo anterior é manualmente checada e os artigos sem dados para 'abstract' e aqueles que não se encaixam nos critérios de inclusão ou apresentam critérios para exclusão são excluídos da tabela.


> Critérios de inclusão: População que engloba crianças, adolescentes ou adultos com transtorno do espectro autista (TEA). Artigos sobre uso de video games em português, inglês ou espanhol. Artigos dos últimos 20 anos.


> Critérios de exclusão: População que não TEA. Outras línguas que não português, inglês ou espanhol. Artigos que não tem como assunto video games. Artigos sem abstracts disponíveis.




SUMARIZAÇÃO DOS ABSTRACTS UTILIZANDO DEEP LEARNING

Os abstracts são sumarizados de forma automatizada utilizando um modelo de deep learning de arquitetura 'transformers' da google o 'Pegasus x-sum'. Os abstracts sumarizados em uma frase são salvos em um arquivo/tabela .csv.  

In [ ]:
#leitura de csv com titulo e abstract
df=pd.read_csv('/content/drive/MyDrive/TRABALHO ABENAPI/1TEAgames-abstracts.csv', usecols= ['Title','Abstract'])
print(df)

                                                Title  \
0   A preliminary study into internet related addi...   
1   Video Game Use, Aggression, and Social Impairm...   
2   I can actually do it without any help or someo...   
3   [Formula: see text]Is active video gaming asso...   
4   A Mobile Game Platform for Improving Social Co...   
..                                                ...   
70  Using computerized games to teach face recogni...   
71  Designing affective video games to support the...   
72  Engagement with electronic screen media among ...   
73  A Virtual Reality Training Application for Adu...   
74                             Helping children play.   

                                             Abstract  
0   In recent decades, studies have investigated a...  
1   We used parent report data to investigate vide...  
2   Research into autistic adolescents' engagement...  
3   Active video gaming (AVG) is a way that childr...  
4   BACKGROUND: Many children with 

In [ ]:
df['Text']=df["Title"].map(str) + " " + df["Abstract"]
text=df['Text'].tolist()
text2=[]
for item in text:
    item2=str(item)
    text2.append(item2)

In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:00


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = text2

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

summary=[]
for text in src_text:
  batch = tokenizer(text, truncation=True, padding="longest", return_tensors="pt").to(device)
  translated = model.generate(**batch)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  summary.append(tgt_text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
myFile = open('demo_file.csv', 'w')
writer = csv.writer(myFile)
writer.writerow(['Sumario'])
for data_list in summary:
    writer.writerow(data_list)
myFile.close()
myFile = open('demo_file.csv', 'r')
print("The content of the csv file is:")
print(myFile.read())
myFile.close()

PROCESSAMENTO DE LINGUAGEM NATURAL E FREQUÊNCIAS

São utilizadas técnicas de processamento de linguagem natural aplicadas as colunas da tabela dos artigos. São calculados as frequências das palavras na coluna Autores para auxiliar na identificação dos autores com o maior número de publicações sobre o tema. O mesmo é feito para a coluna 'Journal', 'Informação
de autores', título e abstracts.

A partir da frequência de palavras da coluna 'abstract' (com a exclusão de stopwords definidas) é montado uma nuvem de palavras que pode apontar para temas mais investigados pelos artigos pubmed sobre video games e o espectro autista.

